# Pricing Caps in QuantLib

In [9]:
import QuantLib as ql 
import pandas as pd

We initialize by setting the valuation date and defining our forecasting and discounting curve to be used in pricing.

In [11]:
# Begin by setting the valuation date of which the cap and the floor should be priced at
ql.Settings.instance().evaluationDate = ql.Date(1, 1, 2022)
# Then we initialize the curve we want to use for discounting and forecasting
dfs = [1, 0.965, 0.94]  # discount factors
dates = [
    ql.Date(1, 1, 2022),
    ql.Date(1, 1, 2023),
    ql.Date(1, 1, 2024),
]  # maturity dates of the discount factors
day_counter = ql.Actual360()
# Note that we will not strip a curve here, but simply use the discount factors and the dates defined above
# By default QuantLib DiscountCurve will log linearly interpolate between the points.
curve = ql.DiscountCurve(dates, dfs, day_counter)
# The curve will note be linked in case we want to update the quotes later on
ts_handle = ql.YieldTermStructureHandle(curve)


The next step involves creating an schedule of dates for which the optionlets of the caps will mature at. 

In [12]:
start_date = ql.Date(1, 1, 2022)
end_date = start_date + ql.Period(12, ql.Months)

# We define the schedule of the cap and floor
schedule = ql.Schedule(
    start_date,                 # Start date of payments
    end_date,                   # End date of payments
    ql.Period(3, ql.Months),    # frequency of payments
    ql.Sweden(),                # Calendar for adjusting for holidays
    ql.ModifiedFollowing,       # Business convention for adjusting for holidays
    ql.ModifiedFollowing,       # Business convention for adjusting for holidays
    ql.DateGeneration.Backward, # Date generation rule for generating the schedule
    False,                      # End of month rule
)

# Create a custom index to track the payments correctly, specifically fixing days.
custom_index= ql.IborIndex(
    "MyIndex",
    ql.Period("3m"),
    0,
    ql.SEKCurrency(),
    ql.Sweden(),
    ql.ModifiedFollowing,
    False,
    ql.Actual360(),
    ts_handle,
)


The last step is to define the pricing engine to use for pricing. We can choose between:

- BlackCapFloorEngine
- BachelierEngine
- AnalyticCapFloorEngine
- TreeCapFloorEngine

In this example we will precede with BlackCapFloorEngine.

In [14]:
# As you have noted by now, the pricing of caps and floors involves creating a floating leg
ibor_leg = ql.IborLeg([1e6], schedule, custom_index)
strike = [0.025]
cap = ql.Cap(ibor_leg, strike)

# The final step is to define a volatility surface, we will use a constant volatility for simplicity
volatility = ql.QuoteHandle(ql.SimpleQuote(0.5))

# Input our discounting and forecasting curve together with our volatility surface to the engine
engine = ql.BlackCapFloorEngine(ts_handle, volatility)
cap.setPricingEngine(engine)
print(cap.NPV())

10831.583434218297


At last we want to show our results of the seperate optionlets.

In [21]:
schedule_dates = schedule.dates()

pd.DataFrame({
    'price': cap.optionletsPrice(),
    'discount_factor': cap.optionletsDiscountFactor(),
    'cap_rate': cap.capRates(),
    'atm_forward': cap.optionletsAtmForward(),
    'std_dev': cap.optionletsStdDev(),
    'accrual_start': schedule_dates[:-1],
    'accrual_end' : schedule_dates[1:]
})

,price,discount_factor,cap_rate,atm_forward,std_dev,accrual_start,accrual_end
0,2493.450264,0.991254,0.025,0.035290,0.037012,"January 3rd, 2022","April 1st, 2022"
1,2625.359083,0.982488,0.025,0.035296,0.248282,"April 1st, 2022","July 1st, 2022"
2,2846.309041,0.973515,0.025,0.035301,0.352097,"July 1st, 2022","October 3rd, 2022"
3,2866.465047,0.964931,0.025,0.035193,0.434000,"October 3rd, 2022","January 2nd, 2023"
